In [1]:
import os
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.utils import *
from fgc_support_retri import config
from evaluation.eval import eval_sp_fgc, _update_sp

In [3]:
def print_sent_ie(sp_i, d):
    print("sentence{}".format(sp_i))
    print(d['SENTS'][sp_i]['text'])
    for e in d['SENTS'][sp_i]['IE']['NER']:
        print(e)

In [4]:
def get_sent_ie(sp_i, d):
    out_string = ""
    out_string += "sentence{}".format(sp_i) + '\n'
    out_string += d['SENTS'][sp_i]['text'] + '\n'
    for e in d['SENTS'][sp_i]['IE']['NER']:
        out_string += str(e) + '\n'
    return out_string

In [5]:
def eval_from_threshold(data, threshold=0.5):
    all_sp_predictions = []
    all_items = []
    all_answer_sp = []
    for d in tqdm(data):
        for q in d['QUESTIONS']:
            if not q['SHINT']:
                continue
            sp_preds = []
            max_i = 0
            max_score = 0
            assert len(q['sp_scores']) == len(d['SENTS'])
            for sp_i, sp_score in enumerate(q['sp_scores']):
                assert sp_score >= 0
                if sp_score >= threshold:
                    sp_preds.append(sp_i)
                if sp_score > max_score:
                    max_i = sp_i
            if not sp_preds:
                sp_preds.append(max_i)
            q['sp'] = sp_preds
            all_sp_predictions.append(sp_preds)
            all_items.append(q['SHINT'])
            all_answer_sp.append(q['answer_sp'])
    return all_items, all_sp_predictions, all_answer_sp

In [7]:
data = json_load("prediction/entity_revise.json")

In [8]:
len(data)

193

In [9]:
get_SHINT(data)

100%|██████████| 193/193 [00:00<00:00, 15094.74it/s]

{'text': '不具入侵能力但失控繁殖的细胞，称为良性肿瘤', 'start': 492, 'end': 513}
{'text': '牠们会在夜晚成群的爬出洞穴，循著海浪的声音，利用海面光线的导引，快速的爬向大海的家', 'start': 431, 'end': 472}
{'text': '企鹅是生长在极寒冷的冰天雪地里，被称为不会飞的鸟，牠的翅膀像鱼鳍，能跟海豚一般在碧波里游泳，', 'start': 58, 'end': 104}
{'text': '企鹅是生长在极寒冷的冰天雪地里，被称为不会飞的鸟，牠的翅膀像鱼鳍，能跟海豚一般在碧波里游泳，却不能像海鸟一样飞翔在蔚蓝的天空。', 'start': 58, 'end': 121}


In [10]:
get_answer_sp(data, force=True)

100%|██████████| 193/193 [00:00<00:00, 40477.06it/s]


In [11]:
all_items, all_sp_predictions, all_answer_sp = eval_from_threshold(data, threshold=0.5)

100%|██████████| 193/193 [00:00<00:00, 45768.12it/s]


In [12]:
metrics = eval_sp_fgc(all_items, all_sp_predictions)

{'sp_em': 0.2, 'sp_prec': 0.604, 'sp_recall': 0.631, 'sp_f1': 0.567}


In [13]:
metrics = eval_sp_fgc(all_answer_sp, all_sp_predictions)

{'sp_em': 0.137, 'sp_prec': 0.373, 'sp_recall': 0.595, 'sp_f1': 0.423}


In [14]:
len(all_answer_sp)

190

In [16]:
def find_q(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q)

In [17]:
def print_analysis_from_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q['QID'])
                print(q['QTEXT_CN'])
                print("atype:{}".format(q['ATYPE']))
                print("SHINT:{}".format(q['SHINT']))
                print("sp:{}".format(q['sp']))
                print("answer:{}".format(q['ANSWER']))
                all_set = set(q['SHINT']) | set(q['sp'])
                for sp_i in range(min(all_set), max(all_set) + 1):
                    print_sent_ie(sp_i, d)
                    print()

In [18]:
print_analysis_from_qid('D105Q01', data)

D105Q01
全民健保于哪一年实施?
atype:Date-Duration
SHINT:[0, 1, 36]
sp:[4, 36]
answer:[{'ATEXT': '1995年', 'ATEXT_CN': '1995年', 'ATOKEN': [{'text': '1995年', 'text_cn': '1995年', 'start': 510, 'end': 515}]}]
sentence0
全民健康保险，

sentence1
一般简称为「全民健保」或「健保」，

sentence2
是一种强制性保险的福利政策，
{'id': 'D0-S0-M0', 'string': '一', 'type': 'NUMBER', 'char_b': 1, 'char_e': 2}

sentence3
法源是依据《中华民国宪法增修条文》所实施的全民医疗保险制度。
{'id': 'D0-S0-M0', 'string': '中华民国宪法增修条文', 'type': 'MISC', 'char_b': 6, 'char_e': 16}

sentence4

在全民健康保险实施以前，

sentence5
台湾按照职业别分别实施军人保险（军保）、公教人员保险（公保）、劳工保险（劳保）、农民健康保险（农保），
{'id': 'D0-S0-M0', 'string': '台湾', 'type': 'STATE_OR_PROVINCE', 'char_b': 0, 'char_e': 2}
{'id': 'D0-S0-M1', 'string': '公保', 'type': 'PER', 'char_b': 27, 'char_e': 29}
{'id': 'D0-S0-M2', 'string': '农民', 'type': 'TITLE', 'char_b': 40, 'char_e': 42}

sentence6
并有保险业各自承保的健康保险。

sentence7
 1986年5月，
{'id': 'D0-S0-M0', 'string': '1986年5月', 'type': 'DATE', 'char_b': 1, 'char_e': 8}

sentence8
行政院核定的「中华民国台湾经济长期展望」中提到以2000年为全民健保开办目标年，
{'id': 'D

In [19]:
find_q('D033Q04', data)

{'QID': 'D033Q04', 'QTYPE': '進階題', 'ATYPE': 'Object', 'AMODE': ['Single-Span-Extraction'], 'QTEXT': '「北美自由貿易協議2.0」的法文簡稱為何?', 'QTEXT_CN': '「北美自由贸易协议2.0」的法文简称为何?', 'SENTS': [{'text': '「北美自由贸易协议2.0」的法文简称为何?', 'start': 0, 'end': 21, 'IE': {'NER': [{'id': 'D0-S0-M0', 'string': '北美', 'type': 'LOC', 'char_b': 1, 'char_e': 3}, {'id': 'D0-S0-M1', 'string': '2.0', 'type': 'NUMBER', 'char_b': 9, 'char_e': 12}, {'id': 'D0-S0-M2', 'string': '法文', 'type': 'DEMONYM', 'char_b': 14, 'char_e': 16}], 'COREF': {}, 'RELATION': [], 'TOKEN': [{'word': '「', 'char_b': 0, 'char_e': 1, 'pos': 'PU'}, {'word': '北美', 'char_b': 1, 'char_e': 3, 'pos': 'NR'}, {'word': '自由', 'char_b': 3, 'char_e': 5, 'pos': 'JJ'}, {'word': '贸易', 'char_b': 5, 'char_e': 7, 'pos': 'NN'}, {'word': '协议', 'char_b': 7, 'char_e': 9, 'pos': 'NN'}, {'word': '2.0', 'char_b': 9, 'char_e': 12, 'pos': 'CD'}, {'word': '」', 'char_b': 12, 'char_e': 13, 'pos': 'PU'}, {'word': '的', 'char_b': 13, 'char_e': 14, 'pos': 'DEC'}, {'word': '法文', 'char_b': 14, '

In [20]:
data[100]

{'DID': 'D201',
 'QUESTIONS': [{'QID': 'D201Q02',
   'QTYPE': '進階題',
   'ATYPE': 'YesNo',
   'AMODE': ['YesNo'],
   'QTEXT': '內政部所發布的新聞稿是否享有著作權?',
   'QTEXT_CN': '内政部所发布的新闻稿是否享有著作权?',
   'SENTS': [{'text': '内政部所发布的新闻稿是否享有著作权?',
     'start': 0,
     'end': 18,
     'IE': {'NER': [{'id': 'D0-S0-M0',
        'string': '内政部',
        'type': 'LOC',
        'char_b': 0,
        'char_e': 3}],
      'COREF': {},
      'RELATION': [],
      'TOKEN': [{'word': '内政部', 'char_b': 0, 'char_e': 3, 'pos': 'NN'},
       {'word': '所', 'char_b': 3, 'char_e': 4, 'pos': 'MSP'},
       {'word': '发布', 'char_b': 4, 'char_e': 6, 'pos': 'VV'},
       {'word': '的', 'char_b': 6, 'char_e': 7, 'pos': 'DEC'},
       {'word': '新闻稿', 'char_b': 7, 'char_e': 10, 'pos': 'NN'},
       {'word': '是否', 'char_b': 10, 'char_e': 12, 'pos': 'AD'},
       {'word': '享有', 'char_b': 12, 'char_e': 14, 'pos': 'VV'},
       {'word': '著作权', 'char_b': 14, 'char_e': 17, 'pos': 'NN'},
       {'word': '?', 'char_b': 17, 'char_e': 18, 'po

In [22]:
d = data[100]
for q in d['QUESTIONS']:
    print(q['QID'])
    print(q['QTEXT_CN'])
    print("atype:{}".format(q['ATYPE']))
    print("SHINT:{}".format(q['SHINT']))
    print("answer_sp:{}".format(q['answer_sp']))
    print("sp:{}".format(q['sp']))
    print()
    all_set = set(q['SHINT']) | set(q['sp'])
    for sp_i in range(min(all_set), max(all_set) + 1):
        print(q['sp_scores'][sp_i])
        print(get_sent_ie(sp_i, d))

D201Q02
内政部所发布的新闻稿是否享有著作权?
atype:YesNo
SHINT:[7, 8, 13]
answer_sp:[7, 8, 13]
sp:[13]

0.23751510679721832
sentence7
不在此限。
第九条下列各款不得为著作权之标的，
{'id': 'D0-S1-M0', 'string': '第九', 'type': 'ORDINAL', 'char_b': 6, 'char_e': 8}

0.006182305049151182
sentence8
依宪法、法律命令或公文。

0.000957159383688122
sentence9
二、中央或地方机关就钱款著作做成之翻译误或编辑误。
{'id': 'D0-S0-M0', 'string': '二', 'type': 'NUMBER', 'char_b': 0, 'char_e': 1}
{'id': 'D0-S0-M1', 'string': '编辑', 'type': 'TITLE', 'char_b': 21, 'char_e': 23}

0.002141648903489113
sentence10

三标语及通用之符号名词、公视术表表格不测或实力是单纯为传达事实之新闻报导所作成之语文著作。
{'id': 'D0-S0-M0', 'string': '三', 'type': 'NUMBER', 'char_b': 1, 'char_e': 2}
{'id': 'D0-S0-M1', 'string': '公视', 'type': 'ORG', 'char_b': 13, 'char_e': 15}

0.00025973509764298797
sentence11

依法令举行之各类考试试题积习备用实体。

0.0006104119238443673
sentence12

前项第一款所称公文，
{'id': 'D0-S0-M0', 'string': '第一', 'type': 'ORDINAL', 'char_b': 3, 'char_e': 5}

0.06946363300085068
sentence13
包括公务员于职务上草拟之文告、讲稿、新闻稿及其他文书。
{'id': 'D0-S0-M0', 'string': '公务员', 'type

In [28]:
len(data)

193

In [31]:
def write_analysis(file_path, data):
    f = open(file_path, 'w')
    for d in data:
        for q in d['QUESTIONS']:
            if not q['SHINT']:
                continue
            f.write(q['QID'] + '\n')
            f.write(q['QTEXT_CN'] + '\n')
            f.write("atype:{}".format(q['ATYPE']) + '\n')
            f.write("SHINT:{}".format(q['SHINT']) + '\n')
            f.write("answer_sp:{}".format(q['answer_sp']) + '\n')
            f.write("sp:{}".format(q['sp']) + '\n')
            f.write('\n')
            all_set = set(q['SHINT']) | set(q['sp'])
            for sp_i in range(min(all_set), max(all_set) + 1):
                f.write(str(q['sp_scores'][sp_i]) + '\n')
                f.write(get_sent_ie(sp_i, d))
    f.close()

In [32]:
write_analysis('prediction/entitiy_revise.analysis', data)